In [1]:
import requests
import pandas as pd
import numpy as np
import sddk
import spacy
import zipfile
import io
import re

In [2]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [3]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_cleaned_dated_2020-12-11.json", "df", conf)

In [4]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,...,clean_text_interpretive_sentence,clean_text_pythia,raw_date,dating_dict,not_before,not_after,or,date_tags,certainty,link
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,{},12,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκε...,c. 510-500 a.,"{'or': None, 'not_before': -511, 'not_after': ...",-511.0,-499.0,None,"[range, phase, ca]",None,None
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,{},14,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,...,"․․ ․․․ν βολ ․ ․․ α ⋮ ℎεκον σιον, γνοσθε͂ι δὲ ν...",ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄλ...,c. 500 a.,"{'or': None, 'not_before': -505, 'not_after': ...",-505.0,-495.0,None,"[exact, phase, ca]",None,None
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,{},13,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,...,αρι ℎερακλειο․․․․ αρο τιθέναι τὸς ἀθλοθέτας τρ...,αρι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκον...,490-480 a.,"{'or': None, 'not_before': -490, 'not_after': ...",-490.0,-480.0,None,[range],None,None
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,{},56,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,...,․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․․ ․․․․․․․․․...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,485/4 a.,"{'or': None, 'not_before': -485, 'not_after': ...",-485.0,-484.0,None,[range],None,None
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,{},6,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,...,ἔδοχσεν ⋮ τε͂ι βολε͂ι ⋮ καὶ το͂ι δέμοι ⋮ ℎότε ...,ἔδοχσεν τει βολει καὶ τοι δέμοι ὅτε παραιβάτες...,c. 500 a.,"{'or': None, 'not_before': -505, 'not_after': ...",-505.0,-495.0,None,"[exact, phase, ca]",None,None


In [5]:
len(PHI[PHI["clean_text_pythia"] ==  PHI["clean_text_interpretive_word"]])

1144

In [6]:
def extend_pythia(row):
    pythia = row["clean_text_pythia"]
    if pythia == "":
        pythia = row["clean_text_interpretive_word"]
    return pythia

PHI["clean_text_pythia_ext"] = PHI.apply(lambda row: extend_pythia(row), axis=1)

In [8]:
### to do below some matching, we have to take care of appropriate encoding
from unicodedata import normalize
def normalize_string(string):
    try:
        return normalize("NFC", string)
    except: 
        return ""
PHI["string_normalized"] = PHI["clean_text_pythia_ext"].apply(normalize_string)

In [25]:
# sentences
# add column with sentences
v_ud = "’"
v_agt = "ʼ"

def get_sentences(string):
    splits = re.split(r"(\.|\·|\:|\;|\?|\!)" , string)
    sentences = []
    for n in range(0, len(splits), 2):
        try:
            sentence = splits[n]+splits[n+1]
            sentence = "".join([c for c in sentence if re.match("[\w|\s|\.|\,|\·|\:|\;|\?|\!]", c)]) 
            sentence = re.sub("^\s*", "", sentence)
            sentence = re.sub(v_ud, v_agt, sentence)
            sentences.append(sentence)
        except:
            sentence = splits[n]
            sentence = re.sub("^\s*", "", sentence)
            if len(sentence) > 0:
                sentences.append(sentence)
            break
    return sentences

In [27]:
PHI["sents"] = PHI["clean_text_pythia_ext"].apply(get_sentences)

In [28]:
PHI["sents_N"] = PHI["sents"].apply(lambda x: len(x))

In [29]:
len(PHI[PHI["sents_N"] > 1])

22321

In [30]:
len(PHI[PHI["sents_N"] > 2])

11125

In [32]:
PHI[PHI["sents_N"] > 2]["sents"].tolist()[:5]

[['δραχμεισι τες τὸς ιο μενος δεμο τον πόλεον δοκει ἀνατιθα ἐάν τις τον ον ἒ ὁσαλ ο ἒ ενορ νας ἵνα αντοιν σ τον πόλεον ταυτ χρε μεμβολ εχθεοχε ν δενλο ἐὰν δὲ μέ ἑκαστοσε αιν ιμιας πε ετατεν ιαο λπο σειανλλαμ ολει καὶ μὲ νεοτερ χρέσθο τοι ἱεροι ἐὰν δὲ ι μὲ χρέσθο ἐὰν δὲ ἰ κατὰ ταὐτὰ ταῦτα ἐὰν πλεῖστον κατὰ τὲν δύναμιν πρᾶχσαι δ ἔκπραχσιν ἐὰν δὲ μὲ ἐγδοι τὸν ὀφλόντα μὲ χρέσθο τοι ἱεροι ἐὰν ἀμφισβετοσι μὲ κλεθεναι ἐμ πόλει εν ἐλθοσαν ἀδικ ὕστερον ἑ βολὲ α ι τὸν ἀθεναῖον μὲ ἐκ γες πο τούτον τον πόλεον μεδὲ ἁμος βιᾶσθαι ἐὰν μὲ δίκεν ὀφλόντα ἐπιχορίαν ἒ ἐς πολεμίος λεφθέντα ἕτις δ ἂν τομ πόλεον μὲ ἐθέλει δίκας διδόναι καὶ δέχεσθαι ἀθεναίοισιν ἀπὸ χσυμβολον.',
  'ια ντο τος τὰ μὲν ἁκόσια ἁπλει τὰ δὲ ἡ εκόσια διπλει σπονδὰς εἶναι τοῖσι μύστεσιν καὶ τοῖς ἐπόπτεισιν καὶ τοῖς ἀκολούθοισιν καὶ χρέμασιν τον ὀθνείον καὶ ἀθεναῖοισιν ἅπασιν ἄρχεν δὲ τὸν χρόνον τον σπονδον το μεταγειτνιονος μενὸς ἀπὸ διχομενίας καὶ τὸν βοεδρομιονα καὶ το υανοφσιονος μέχρι δεκάτες ἱσταμένο τὰς δὲ σπονδὰς εἶναι ἐν τεισι 

# Download spacy model from sciencedata, load it etc.

In [55]:
#
#!curl https://sciencedata.dk//shared/b8f6c317c433385ce79a9531a0fbf658?download -o ../data/spacy_grc_model.zip
#!unzip  -: ../data/spacy_grc_model.zip

In [31]:
nlp = spacy.load("../models/spacy_grc_model_7")
path = "../models/spacy_grc_model_7/grc_lemmatization.py"
exec(open(path).read())
nlp.add_pipe(grc_doc_lemmatizer, "grc_doc_lemmatizer", after="tagger")

In [ ]:
%%time
def try_apply_nlp(sents):
    try: return apply_nlp(sents)
    except: return [nlp("")]

## apply it on the normalized string
PHI_docs_series = PHI["string_normalized"].apply(try_apply_nlp)

In [ ]:
def extraxt_lem_sents(sentences):
    lemmatized_sentences = []
    for sent in sentences:
        lemmata = []
        for token in doc:
            if token.pos_ in ["NOUN", "PROPN", "ADJ", "VERB"]:
                lemmata.append(token.lemma_)
        lemmatized_sentences.append(lemmata)
    return lemmatized_sentences

In [ ]:
lem_sents_series = PHI_docs_series.apply(extraxt_lem_sents)

In [ ]:
lem_sents_series

In [ ]:
%%time
PHI["lem_sents"] = lem_sents_series
#PHI["lemmata"] = PHI["string_normalized"].apply(lambda x: lemmatize_string(x, all_lemmata=False, filter_by_postag=["n", "a", "v"], involve_unknown=True))

In [ ]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
PHI["lemmata"] = PHI["lem_sents"].apply(flat_lemmata)

In [ ]:
sddk.write_file("SDAM_data/PHI/PHI_dated_lemmatized_20201211.json", PHI, conf)

In [ ]:
PHI.columns

# Reading the data back

In [119]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_dated_lemmatized_20201211.json", "df", conf)

In [ ]:
def count_words(x):
    try:
        return len(x.split())
    except:
        return 0
PHI["clean_text_interpretive_word"].apply(count_words).sum() # previously 4469462, then 

In [ ]:
PHI["lemmata"].apply(lambda x: len(x)).sum() # previously 2743932, 2846806, 2867815